## Recuperação da Informação
####Professor: Leandro Balby
####Aluno: Lucas Barros Rocha - 115111579

In [23]:
#bibliotecas

import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import time
import re
import heapq

nltk.download('stopwords')

#palavras a serem desconsideradas
tknz = RegexpTokenizer(r'(\b[A-Za-z][a-z]{2,27}\b)')
stopwords = nltk.corpus.stopwords.words('portuguese')

#instalação de biblioteca não nativa no colab 
!pip install memory_profiler


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Importando os Dados, que são: notícias retiradas por crawler+scraper de portais de notícas online.

In [0]:
result = pd.read_csv('https://raw.githubusercontent.com/lucasbarrosrocha/rec-info/master/dataframes/results.csv')

#### 1. Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.


In [0]:
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

In [0]:
tokens_table = pd.DataFrame()

tokens_table['Word'] = [word for word in indexes.keys()]
tokens_table['Documents'] = [docs for docs in indexes.values()]

tokens_table.to_csv('ranking.csv', encoding='utf-8', index=False)

#tokens_table

#### 2. Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

In [0]:

def doc_a_time(query, indexes, k):
    start = time.time()
    
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for doc in range(1, len(result.text)+1):
      doc_score = 0
      for index in query_indexes:
          for i in index:
              if i[0] == doc:
                  doc_score += i[1]
                  break
      if doc_score != 0:
        heapq.heappush(rank, (doc_score, doc))
    
    end = time.time() - start
    return heapq.nlargest(k, rank), end

In [0]:

def term_a_time(query, indexes, k):
    start = time.time()
    
    a = {} # Accumulator HashTable
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    end = time.time() - start
    return heapq.nlargest(k, rank), end

In [0]:
queries = ["brasil", "educação", "governo", "futebol", "economia", "política", "cultura"]

In [0]:

k = 10
results_doc = []
results_term = []
times_doc = []
times_term = []

for query in queries:
  score_doc, time_doc = doc_a_time(query, indexes, k)
  results_doc.append(score_doc)
  times_doc.append(time_doc)
  
  score_term, time_term = term_a_time(query, indexes, k)
  results_term.append(score_term)
  times_term.append(time_term)

In [37]:
queries_df = pd.DataFrame()
queries_df['Query'] = queries
queries_df['Document at a time'] = results_doc
queries_df['Term at a time'] = results_term
queries_df['Comparative'] = queries_df['Document at a time'] == queries_df['Term at a time']
queries_df.index+=1
queries_df

,Query,Document at a time,Term at a time,Comparative
1,brasil,"[(47, 151), (15, 166), (11, 19), (10, 26), (9,...","[(47, 151), (15, 166), (11, 19), (10, 26), (9,...",True
2,educação,[],[],True
3,governo,"[(15, 173), (13, 166), (10, 248), (10, 115), (...","[(15, 173), (13, 166), (10, 248), (10, 115), (...",True
4,futebol,"[(8, 242), (7, 118), (7, 47), (6, 115), (5, 21...","[(8, 242), (7, 118), (7, 47), (6, 115), (5, 21...",True
5,economia,"[(10, 138), (8, 125), (6, 127), (6, 69), (6, 3...","[(10, 138), (8, 125), (6, 127), (6, 69), (6, 3...",True
6,política,[],[],True
7,cultura,"[(10, 124), (4, 158), (3, 248), (3, 165), (2, ...","[(10, 124), (4, 158), (3, 248), (3, 165), (2, ...",True


In [18]:

queries_tempo_df = pd.DataFrame()
queries_tempo_df['Average time: Document at a time (ms)'] = times_doc
queries_tempo_df['Average time: Term at a time (ms)'] = times_term
queries_tempo_df.index+=1
queries_tempo_df

,Average time: Document at a time (ms),Average time: Term at a time (ms)
1,0.001626,0.000077
2,0.000516,0.000021
3,0.001372,0.000070
4,0.000386,0.000017
5,0.000746,0.000034
6,0.001518,0.000062
7,0.000435,0.000017


In [24]:
%load_ext memory_profiler

print('Documento por vez uso de memória:')
%memit doc_a_time(queries[0], indexes, k)

print('Termo por vez uso de memória:')
%memit term_a_time(queries[0], indexes, k)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Documento por vez uso de memória:
peak memory: 205.89 MiB, increment: 0.00 MiB
Termo por vez uso de memória:
peak memory: 205.89 MiB, increment: 0.00 MiB


#### 3. Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

In [0]:
def conj_query(query, indexes, k):
  
  query_indexes = [] # Array
  rank = [] # HeapQueue, implementation of PriorityQueue
  
  for word in query.split(" "):
    if word in indexes.keys():
        query_indexes.append(indexes[word])
  
  all_indexes = [item for sublist in query_indexes for item in sublist]
  all_indexes.sort() # Sorted Array of all indexes from query terms
  
  for i in range(len(all_indexes)):
    doc_score = 0
    d = all_indexes.pop()
    repeat = 1
    for index in all_indexes:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

In [0]:
queries_conj = ["novos ajustes", "futebol brasileiro", "tudo sobre", "esportes estão", "jair bolsonaro"]

In [0]:

k = 10
results_conj = []

for query in queries_conj:
  score_conj = conj_query(query, indexes, k)
  results_conj.append(score_conj)

In [61]:

conj_queries_df = pd.DataFrame()
conj_queries_df['Query'] = queries_conj
conj_queries_df['Results'] = results_conj
conj_queries_df.index+=1
conj_queries_df

,Query,Results
1,novos ajustes,"[(2, 121), (2, 107)]"
2,futebol brasileiro,"[(10, 115), (7, 55), (5, 50), (3, 121), (3, 76..."
3,tudo sobre,"[(17, 151), (13, 6), (12, 211), (12, 69), (11,..."
4,esportes estão,[]
5,jair bolsonaro,"[(52, 151), (48, 207), (39, 166), (13, 216), (..."
